In [1]:
import os
import pickle as pkl
from collections import defaultdict

In [2]:
script_dir = os.getcwd()
os.chdir(os.path.join("..", "results", "ser_perf"))

ser_perf_dir = os.getcwd()
os.chdir(script_dir)

model_perf = os.path.join(ser_perf_dir, "performance_titanet_en_ch_40ep_eval_sp_7_en_original_data_False.pkl")

In [3]:
training_langs = ["en", "ch", "en_ch"]
eval_sps = [7, 8]
# original_data = [False, True]
titanet_perf = {"original data": dict(), "remixed data": dict()}
wav2vec2_perf = {"original data": dict(), "remixed data": dict()}
perfs = {"titanet": titanet_perf, "wav2vec2": wav2vec2_perf}
data_settings = [("remixed data", False), ("original data", True)]

In [4]:
for model in perfs:
    for data_setting, data_bool in data_settings:
        for training_lang in training_langs:
            labels_correct = {"en": defaultdict(list), "ch": defaultdict(list)}
            for eval_sp in eval_sps:
                for eval_lang in training_lang.split("_"):
                    perf_file = f"performance_{model}_{training_lang}_40ep_eval_sp_{eval_sp}_{eval_lang}_original_data_{data_bool}.pkl"
                    perf_path = os.path.join(ser_perf_dir, perf_file)
                    with open(perf_path, "rb") as infile:
                        perf = pkl.load(infile)
                        for test_lang in ["en", "ch"]:
                            for label in [0,1,2,3]:
                                labels_correct[test_lang][label] += perf["test perf"][test_lang]["labels correct"][label]
            perfs[model][data_setting][training_lang] = labels_correct

In [5]:
perf["test perf"]["en"]["labels correct"].keys()
perf["test perf"]["en"]["labels correct"][0]
perf["test perf"].keys()

dict_keys(['en', 'ch'])

In [6]:
idx_to_label = {0: "happy", 1: "angry", 2: "sad", 3: "neutral"}

In [8]:
def print_per_label_perf(model, model_perf):
    code_to_lang = {"en": "English", "ch": "Chinese"}
    print(f"{model}")
    # print(f"{model} {data_setting}")
    print(f"{'model':<7}   {'en':>5} {'ch':>5} {'en/ch':>5}{'en':>5} {'ch':>5} {'en/ch':>5}")
    print(f"{'dataset':<7}   {'Original':^20}{'Remixed':^20}")
    for lang in ["en", "ch"]:
        print(f"\multicolumn{{7}}{{c}}{{{code_to_lang[lang]}}} \\\\")
        print("\hline")
        for label in [0,1,2,3]:
            str_label = idx_to_label[label]
            print(f"{str_label:<7}   ", end="")
            for data_setting, _ in data_settings[::-1]:
                for training_lang in training_langs:
                    preds = model_perf[data_setting][training_lang][lang][label]
                    score = sum(preds)/len(preds)*100
                    print(f"& \\textcolor{{black!{round((score+20)/1.2)}}}{{{score:.1f}}}", end=" ")
            print("\\\\")
        print("\hline")

for model in perfs:
    # for data_setting, _ in data_settings:
    print_per_label_perf(model, perfs[model])
    print()

titanet
model        en    ch en/ch   en    ch en/ch
dataset         Original            Remixed       
\multicolumn{7}{c}{English} \\
\hline
happy     & \textcolor{black!95}{93.8} & \textcolor{black!57}{48.0} & \textcolor{black!95}{93.9} & \textcolor{black!48}{37.0} & \textcolor{black!47}{36.5} & \textcolor{black!52}{42.3} \\
angry     & \textcolor{black!97}{96.7} & \textcolor{black!50}{40.3} & \textcolor{black!97}{95.9} & \textcolor{black!84}{80.4} & \textcolor{black!28}{13.6} & \textcolor{black!58}{50.0} \\
sad       & \textcolor{black!97}{96.7} & \textcolor{black!67}{60.2} & \textcolor{black!97}{96.6} & \textcolor{black!84}{80.5} & \textcolor{black!70}{64.4} & \textcolor{black!82}{78.5} \\
neutral   & \textcolor{black!98}{97.8} & \textcolor{black!56}{46.7} & \textcolor{black!98}{97.8} & \textcolor{black!55}{46.6} & \textcolor{black!79}{75.0} & \textcolor{black!83}{79.9} \\
\hline
\multicolumn{7}{c}{Chinese} \\
\hline
happy     & \textcolor{black!75}{70.0} & \textcolor{black!99}{98.

In [38]:
def print_perf(model, model_perf):
    # code_to_lang = {"en": "English", "ch": "Chinese"}
    model_to_table_name = {"titanet": "TitaNet", "wav2vec2": "Wav2Vec2.0"}
    table_model = model_to_table_name[model]
    # print(f"{model}")
    # print(f"{model} {data_setting}")
    # print(f"{'model':<7}   {'en':>5} {'ch':>5} {'en/ch':>5}{'en':>5} {'ch':>5} {'en/ch':>5}")
    # print(f"{'dataset':<7}   {'Original':^20}{'Remixed':^20}")
    for training_lang in training_langs:
        # print(f"\multicolumn{{7}}{{c}}{{{code_to_lang[lang]}}} \\\\")
        # print("\hline")
            # str_label = idx_to_label[label]
        print(f"{table_model}-", end="")
        print(f"{training_lang.replace('_', '/').upper()} ", end="")
        for data_setting, _ in data_settings[::-1]:
            for lang in ["en", "ch"]:
                score = []
                for label in [0,1,2,3]:              
                    score += model_perf[data_setting][training_lang][lang][label]
                    # print(f"& {score:.1f}", end=" ")
        #     print("\\\\")
        # print("\hline")
                avg = sum(score)/len(score)*100
                print(f"& {avg:.1f}", end=" ")
        print("\\\\")
    print("\hline")

for model in perfs:
    # for data_setting, _ in data_settings:
    print_perf(model, perfs[model])

TitaNet-EN & 96.2 & 57.9 & 61.1 & 58.8 \\
TitaNet-CH & 48.8 & 98.2 & 47.4 & 79.4 \\
TitaNet-EN/CH & 96.0 & 98.6 & 62.7 & 82.1 \\
\hline
Wav2Vec2.0-EN & 90.3 & 60.4 & 71.3 & 58.7 \\
Wav2Vec2.0-CH & 51.9 & 95.6 & 58.7 & 78.6 \\
Wav2Vec2.0-EN/CH & 88.6 & 94.1 & 71.7 & 77.4 \\
\hline
